# AiPet - Face sync

In [ ]:
!pip install imageio-ffmpeg

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

In [ ]:
import os
os.chdir('first-order-model')

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

from demo import make_animation
from skimage import img_as_ubyte

## Load Model

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='../../input/first-order-motion-model/vox-cpk.pth.tar')

In [ ]:
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

## Face sync

In [ ]:
# select source image & driving video
source_image = imageio.imread('../../input/input-pets/Jerry.jpg')
driving_video = imageio.mimread('../../input/input-talking/Trump.mp4')

# images resize to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

# Image Animation
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

In [ ]:
##save resulting video
#imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])